## Environment Setup
**Make sure to set your runtime to use a GPU by going to `Runtime` -> `Change runtime type` -> `Hardware accelerator` -> `T4 GPU`**

In [1]:
import sys
if 'google.colab' in sys.modules:  # If in Google Colab environment
    # Mount google drive to enable access to data files
    from google.colab import drive
    drive.mount('/content/drive')

    # Installing requisite packages
    !pip install transformers accelerate pymupdf &> /dev/null

    # Change working directory to health
    %cd /content/drive/MyDrive/LLM4BeSci_GSERM2024/day_5

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/LLM4BeSci_GSERM2024/day_5


In [2]:
import fitz
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

## Load Data and Model

In [3]:
# Extract text from PDF
pdf = fitz.open('Against_Justaism.pdf')
text = ""
for page in pdf[:4]:
    text += page.get_text()

text[:1000]  # Display the first 500 characters to verify content extraction

'Against Justaism: A call for more measured\ndiscussions on LLM cognition\nZak Hussain1,2*, Rui Mata1 and Dirk U. Wulff2,1\n1*University of Basel.\n2Max Planck Institute for Human Development.\n*Corresponding author(s). E-mail(s): z.hussain@unibas.ch;\nContributing authors: rui.mata@unibas.ch; wulff@mpib-berlin.mpg.de;\nAbstract\nAre large language models (LLMs) ”just next-token predictors” devoid of cogni-\ntive capacities such as ”thought” or ”understanding”? We argue from a cognitive\nscience perspective that it is far from self-evident that such perspectives hold\ndespite their prominence in current discussions surrounding LLMs. We make our\ncase by critically evaluating different flavors of Justaic reasoning and end with a\ncall for more measured discussions on LLM cognition.\nKeywords: large language models, cognition\nOver 70 years ago, Turing (1950) posed a question that has captivated computer sci-\nentists, cognitive scientists, and philosophers alike: ”Can machines think?”. 

In [4]:
torch.random.manual_seed(42) # For reproducibility

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda", # Use GPU
    torch_dtype=torch.float16, # Use half-precision
    trust_remote_code=True,
    attn_implementation='eager' # For faster inference on T4 GPUs
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 300,  # Maximum number of tokens to generate
    "return_full_text": False, # Return only the generated text
    "do_sample": False # Use greedy decoding
}

In [12]:
def query_paper(query):
    prompt = [
        {"role": "assistant", "content": "What do you want to know about the paper?:\n-------------------\n" + text},
        {"role": "user", "content": query}
        ]
    output = pipe(prompt, **generation_args)
    return output[0]['generated_text']

print(query_paper("What is the title of the paper?"))

 The title of the paper is "Against Justaism: A call for more measured discussions on LLM cognition".


In [13]:
print(query_paper("Return the full abstract from the paper"))

 Against Justaism: A call for more measured discussions on LLM cognition
Zak Hussain1,2*, Rui Mata1 and Dirk U. Wulff2,1
1*University of Basel.
2Max Planck Institute for Human Development.
*Corresponding author(s). E-mail(s): z.hussain@unibas.ch;
Contributing authors: rui.mata@unibas.ch; wulff@mpib-berlin.mpg.de;
Abstract
Are large language models (LLMs) ”just next-token predictors” devoid of cogni-
tive capacities such as ”thought” or ”understanding”? We argue from a cognitive
science perspective that it is far from self-evident that such perspectives hold
despite their prominence in current discussions surrounding LLMs. We make our
case by critically evaluating different flavors of Justaic reasoning and end with a
call for more measured discussions on LLM cognition.
Keywords: large language models, cognition


In [14]:
print(query_paper("Summarise the paper"))

 The paper critiques two flavors of Justaic reasoning in the context of large language models (LLMs) and cognition. The first, anti-simple-objectives Justaism, argues that LLMs, being merely next-token predictors, lack cognitive capacities. The authors counter this by suggesting that simple objectives could still lead to cognition, using humans as an analogy. The second, anti-anthropomorphic Justaism, posits that attributing cognition to machines is a category error. The authors argue against this restrictive view, advocating for conceptual generalization and a more nuanced discussion on extending cognitive capacities to AI systems. The paper also addresses other criticisms of LLM cognition, such as the distinction between semantics and syntax, and the importance of real-world grounding.
